In [1]:
%cd ..

e:\Projects\audioset_coarse_grained_classification


In [2]:

import numpy as np
import torch
from lib.training.training import read_config_from_file
from lib.training.importer import import_scheme

config_file = r'models\audioset\swishnetv2_ws100_bw20_mw16_vf4_adamw2_f\config_input.yaml'
config = read_config_from_file(config_file)
# config['distributed'] = False
config['dataset_path'] = 'G:/datasets/audioset-derived.zip'
SCHEME = import_scheme(config['scheme'])

training = SCHEME(config)
training.load_checkpoint()

Checkpoint loaded from: models/audioset\swishnetv2_ws100_bw20_mw16_vf4_adamw2_f\checkpoint


In [3]:
training.model.eval()
data = training.val_dataset[0]
x = torch.from_numpy(data['log_mfb'][None,...]).cuda()
x = {'log_mfb': x}
with torch.no_grad():
    pred = training.model(x)

print(pred)

100%|██████████| 1568/1568 [00:01<00:00, 1371.84it/s]


tensor([[-6.3940e-03, -8.0824e+00, -5.1053e+00]], device='cuda:0')


# Timings

In [6]:
training.model.eval()
# data = training.val_dataset[0]
# x = torch.from_numpy(data['log_mfb'][None,...]).cuda()
x = torch.randn(1, 100, 64).cuda()
x = {'log_mfb': x}
with torch.no_grad():
    %timeit training.model(x)


1.85 ms ± 40.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
training.model.eval()
# data = training.val_dataset[0]
# x = torch.from_numpy(data['log_mfb'][None,...]).cuda()
x = torch.randn(1, 200, 64).cuda()
x = {'log_mfb': x}
with torch.no_grad():
    %timeit training.model(x)


1.87 ms ± 92.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
training.model.eval()
# data = training.val_dataset[0]
# x = torch.from_numpy(data['log_mfb'][None,...]).cuda()
x = torch.randn(1, 400, 64).cuda()
x = {'log_mfb': x}
with torch.no_grad():
    %timeit training.model(x)


1.87 ms ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
cpu_model = training.model.cpu()
# data = training.val_dataset[0]
# x = torch.from_numpy(data['log_mfb'][None,...]).cuda()
x = torch.randn(1, 100, 64)
x = {'log_mfb': x}
with torch.no_grad():
    %timeit cpu_model(x)

2.15 ms ± 82.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
cpu_model = training.model.cpu()
# data = training.val_dataset[0]
# x = torch.from_numpy(data['log_mfb'][None,...]).cuda()
x = torch.randn(1, 200, 64)
x = {'log_mfb': x}
with torch.no_grad():
    %timeit cpu_model(x)

2.52 ms ± 304 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
cpu_model = training.model.cpu()
# data = training.val_dataset[0]
# x = torch.from_numpy(data['log_mfb'][None,...]).cuda()
x = torch.randn(1, 400, 64)
x = {'log_mfb': x}
with torch.no_grad():
    %timeit cpu_model(x)

2.86 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Ensemble

In [9]:
x['log_mfb'].shape

torch.Size([200, 64])

In [6]:
x

{'log_mfb': tensor([[ -7.8278,  -7.4178,  -7.0903,  ...,  -6.4125,  -6.4606,  -6.4250],
         [ -8.6826,  -8.3862,  -8.5069,  ...,  -8.2381,  -9.7233, -10.1630],
         [ -7.4627,  -7.0683,  -6.9251,  ...,  -8.2676,  -9.3752,  -9.2484],
         ...,
         [ -7.9848,  -7.7456,  -7.5762,  ...,  -7.4857,  -8.7206,  -9.4450],
         [ -8.5150,  -8.0476,  -8.0964,  ...,  -7.8460,  -8.5122,  -9.2318],
         [ -8.6083,  -8.2983,  -8.0087,  ...,  -6.8229,  -7.9620,  -9.2039]],
        device='cuda:0')}

In [6]:
from tqdm import tqdm

training.model.eval()
dataset = training.val_dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False)

preds = []
labels = []
with torch.no_grad():
    for data in tqdm(dataloader):
        x = data['log_mfb'].cuda()
        y = data['label']
        pred = training.model({'log_mfb': x})
        pred = torch.softmax(pred, dim=-1)
        
        preds.append(pred.cpu().numpy())
        labels.append(y.numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)

100%|██████████| 86/86 [00:11<00:00,  7.60it/s]


In [7]:
from sklearn.metrics import accuracy_score

pred_cls = np.argmax(preds, axis=-1)
accuracy = accuracy_score(labels, pred_cls)
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 95.87%


In [8]:
preds

array([[9.1466093e-01, 9.3000876e-03, 7.6038919e-02],
       [9.3254727e-01, 3.7071428e-03, 6.3745573e-02],
       [9.2739975e-01, 1.9905164e-03, 7.0609689e-02],
       ...,
       [9.7287548e-01, 4.7671713e-04, 2.6647929e-02],
       [9.4855469e-01, 3.8754931e-03, 4.7569837e-02],
       [9.5927858e-01, 2.6127147e-03, 3.8108759e-02]], dtype=float32)

In [35]:
from tqdm import tqdm
import numpy as np
import torch
from lib.training.training import read_config_from_file
from lib.training.importer import import_scheme
from collections import defaultdict

all_preds = defaultdict(list)

for ii in range(8):
    config_file = f'models/audioset/vgg16_ws200_ll64_vf4_{ii}/config_input.yaml'
    config = read_config_from_file(config_file)
    # config['distributed'] = False
    config['dataset_path'] = 'G:/datasets/audioset-derived.zip'
    SCHEME = import_scheme(config['scheme'])

    training = SCHEME(config)
    training.load_checkpoint()
    training.model.eval()
    config = training.config
    dataset_config, dataset_class = training.get_dataset_config()
    dataset = dataset_class(**dataset_config, 
                            include_ytid = True, 
                            include_window=True,
                            ytid_keys=[str(k) for k in 
                                        config.val_folds])
    dataset.load_data()
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False)

    # preds = {}
    labels = {}
    with torch.no_grad():
        for data in tqdm(dataloader):
            ytids = data['ytid']
            windows = data['window'].numpy()
            x = data['log_mfb'].cuda()
            y = data['label']
            pred = training.model({'log_mfb': x})
            pred = torch.softmax(pred, dim=-1)
            
            pred = pred.cpu().numpy()
            y = y.numpy()
            
            for ytid, window, pred_, label_ in zip(ytids, windows, pred, y):
                all_preds[ytid, tuple(window)].append((pred_))
                labels[ytid, tuple(window)] = label_


Checkpoint loaded from: models/audioset\vgg16_ws200_ll64_vf4_0\checkpoint


100%|██████████| 86/86 [00:11<00:00,  7.53it/s]


Checkpoint loaded from: models/audioset\vgg16_ws200_ll64_vf4_1\checkpoint


100%|██████████| 86/86 [00:11<00:00,  7.50it/s]


Checkpoint loaded from: models/audioset\vgg16_ws200_ll64_vf4_2\checkpoint


...

100%|██████████| 86/86 [00:11<00:00,  7.51it/s]


Checkpoint loaded from: models/audioset\vgg16_ws200_ll64_vf4_6\checkpoint


100%|██████████| 86/86 [00:11<00:00,  7.52it/s]


Checkpoint loaded from: models/audioset\vgg16_ws200_ll64_vf4_7\checkpoint


100%|██████████| 86/86 [00:11<00:00,  7.50it/s]


In [36]:
y_true = []
y_pred = []
for k, label_ in labels.items():
    preds = np.stack(all_preds[k],0).mean(0)
    pred_cls = np.argmax(preds, axis=-1)
    y_true.append(label_)
    y_pred.append(pred_cls)

In [11]:
ensemble_preds = np.mean(all_preds,axis=0)
ensemble_preds

array([[8.34983647e-01, 6.10195892e-03, 1.58914492e-01],
       [8.66233885e-01, 1.29087386e-03, 1.32475227e-01],
       [8.78881991e-01, 1.23626203e-03, 1.19881764e-01],
       ...,
       [9.77021098e-01, 6.07912429e-04, 2.23710835e-02],
       [9.69656825e-01, 3.35614826e-03, 2.69870516e-02],
       [9.60859239e-01, 2.84049870e-03, 3.63002904e-02]], dtype=float32)

In [37]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 96.20%


In [39]:
y_pred.__len__()

10904